# Apache Spark - Tutorial
-----

The objective of this tutorial lab is to introduce you to the fundamentals of Spark programming, including basic concepts, architecture, and hands-on exercises to get you started with writing Spark applications.


## Spark Environment

In [1]:
# Ensure the dataset is present (creates ./dataset/*.csv)
from downloader import ensure_dataset

ensure_dataset()

Extracting CSVs into ./dataset/ ...
Done. CSVs are in: /uufs/chpc.utah.edu/common/home/u1471478/Documents/CS-6964-Lab-2-Do-Not-Share-with-Students/dataset


PosixPath('/uufs/chpc.utah.edu/common/home/u1471478/Documents/CS-6964-Lab-2-Do-Not-Share-with-Students/dataset')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [3]:
import pandas as pd

In [4]:
spark = SparkSession.builder.appName("spark-tutorial").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/28 15:57:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
sc = spark.sparkContext

Spark RDDs
--

In [6]:
# Load data from CSV files
titles_rdd = sc.textFile("dataset/titles.csv")
credits_rdd = sc.textFile("dataset/credits.csv")

In [7]:
print("Titles RDD:")
print(titles_rdd.take(2))
print("\nCredits RDD:")
print(credits_rdd.take(2))

Titles RDD:


[',id,type,release_year,age_certification,runtime,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score', '0,ts300399,SHOW,1945,TV-MA,51,1.0,,,,0.6,']

Credits RDD:
['person_id,id,name,character,role', '3748,tm84618,Robert De Niro,Travis Bickle,ACTOR']


In [8]:
total_titles = titles_rdd.count()
print("\nTotal number of titles:", total_titles)


Total number of titles: 5851


In [9]:
titles_rdd.take(2)

[',id,type,release_year,age_certification,runtime,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score',
 '0,ts300399,SHOW,1945,TV-MA,51,1.0,,,,0.6,']

Map Transformation: Apply a function to each element of the RDD.

In [10]:
# Map Transformation: Extract names from each line
names_rdd = credits_rdd.map(lambda line: line.split(",")[2])


In [11]:
print("Names extracted from each line:")
for name in names_rdd.take(5):
    print(name)

Names extracted from each line:
name
Robert De Niro
Jodie Foster
Albert Brooks
Harvey Keitel


In [12]:
names_rdd = titles_rdd.map(lambda line: line.split(",")[10])
for name in names_rdd.take(5):
    print(name)

tmdb_popularity
0.6
40.965
10.01
15.461


Filter Transformation: Filter out elements of the RDD based on a condition.

In [13]:
directors_rdd = credits_rdd.filter(lambda line: line.split(",")[4] == "DIRECTOR")
print("\nFiltered Directors Rows (4):")
for director in directors_rdd.take(4):
    print(director)


Filtered Directors Rows (4):
3308,tm84618,Martin Scorsese,,DIRECTOR
17727,tm154986,John Boorman,,DIRECTOR
11475,tm127384,Terry Jones,,DIRECTOR
11473,tm127384,Terry Gilliam,,DIRECTOR


Reduce Action: Aggregate the elements of the RDD using a function.

In [14]:
movies_count = titles_rdd.filter(lambda line: "MOVIE" in line).count()
tv_shows_count = titles_rdd.filter(lambda line: "SHOW" in line).count()
print("Number of movies:", movies_count)
print("Number of TV shows:", tv_shows_count)

Number of movies: 3744
Number of TV shows: 2106


In [15]:
# Calculate the average runtime of movies
total_runtime = titles_rdd.filter(lambda x: x.split(',')[2] == "MOVIE") \
                          .map(lambda x: int(x.split(',')[5])) \
                          .sum()
num_movies = titles_rdd.filter(lambda x: x.split(',')[2] == "MOVIE").count()
average_runtime = total_runtime / num_movies

print("Average runtime of movies:", average_runtime)

Average runtime of movies: 98.21367521367522


###Spark Dataframe

In [16]:
titles_df = spark.read.option("header", "true").csv("dataset/titles.csv")
credits_df = spark.read.option("header", "true").csv("dataset/credits.csv")

In [17]:
# 1. Determine the average runtime of movies
average_runtime_movies_df = titles_df.filter("type = 'MOVIE'").agg({"runtime": "avg"}).collect()[0]["avg(runtime)"]
print("\nAverage runtime of movies (DataFrame):", average_runtime_movies_df)



Average runtime of movies (DataFrame): 98.21367521367522


Filtering: Filter the DataFrame to find movies released after specific_year.

In [18]:
# 2. Find the titles released after a specific year
specific_year = 1980
titles_after_specific_year_df = titles_df.filter(f"release_year > {specific_year}")
print(f"Titles released after {specific_year} (DataFrame):")
titles_after_specific_year_df.show()

Titles released after 1980 (DataFrame):
+---+--------+-----+------------+-----------------+-------+-------+---------+----------+----------+---------------+----------+
|_c0|      id| type|release_year|age_certification|runtime|seasons|  imdb_id|imdb_score|imdb_votes|tmdb_popularity|tmdb_score|
+---+--------+-----+------------+-----------------+-------+-------+---------+----------+----------+---------------+----------+
| 35| ts20681| SHOW|        1989|            TV-PG|     24|    9.0|tt0098904|       8.9|  308824.0|        130.213|     8.301|
| 36|tm155787|MOVIE|        1990|                R|    145|   NULL|tt0099685|       8.7| 1131681.0|         50.387|     8.463|
| 37| tm22327|MOVIE|        1987|                R|    117|   NULL|tt0093058|       8.3|  729274.0|         32.937|       8.1|
| 38|tm180542|MOVIE|        1984|                R|    139|   NULL|tt0087843|       8.3|  345714.0|         28.937|     8.453|
| 39|tm138875|MOVIE|        1989|                R|     96|   NULL|tt00

26/01/28 15:57:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, type, release_year, age_certification, runtime, seasons, imdb_id, imdb_score, imdb_votes, tmdb_popularity, tmdb_score
 Schema: _c0, id, type, release_year, age_certification, runtime, seasons, imdb_id, imdb_score, imdb_votes, tmdb_popularity, tmdb_score
Expected: _c0 but found: 
CSV file: file:///uufs/chpc.utah.edu/common/home/u1471478/Documents/CS-6964-Lab-2-Do-Not-Share-with-Students/dataset/titles.csv


In [19]:
# 3. Filter TV shows with a specific age certification
specific_age_certification_tv_shows_df = titles_df.filter((titles_df.type == "SHOW") & (titles_df.age_certification == "TV-MA"))
print("\nTV shows with age certification 'TV-MA' (DataFrame):")
specific_age_certification_tv_shows_df.show()



TV shows with age certification 'TV-MA' (DataFrame):
+---+--------+----+------------+-----------------+-------+-------+---------+----------+----------+---------------+----------+
|_c0|      id|type|release_year|age_certification|runtime|seasons|  imdb_id|imdb_score|imdb_votes|tmdb_popularity|tmdb_score|
+---+--------+----+------------+-----------------+-------+-------+---------+----------+----------+---------------+----------+
|  0|ts300399|SHOW|        1945|            TV-MA|     51|    1.0|     NULL|      NULL|      NULL|            0.6|      NULL|
| 26| ts45948|SHOW|        1972|            TV-MA|     43|    1.0|tt0202477|       8.1|    2151.0|          1.487|       7.0|
|160| ts21684|SHOW|        1994|            TV-MA|     45|    7.0|tt5582224|       7.8|    3139.0|          7.222|       7.3|
|165| ts28516|SHOW|        2000|            TV-MA|     40|    1.0|tt0289649|       9.0|    2399.0|          8.565|       8.8|
|169| ts22039|SHOW|        1998|            TV-MA|     25|    4.

26/01/28 15:57:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, type, release_year, age_certification, runtime, seasons, imdb_id, imdb_score, imdb_votes, tmdb_popularity, tmdb_score
 Schema: _c0, id, type, release_year, age_certification, runtime, seasons, imdb_id, imdb_score, imdb_votes, tmdb_popularity, tmdb_score
Expected: _c0 but found: 
CSV file: file:///uufs/chpc.utah.edu/common/home/u1471478/Documents/CS-6964-Lab-2-Do-Not-Share-with-Students/dataset/titles.csv


##Join Operations
 joins can be performed on spark dataframes using the join method

In [20]:
# Join titles and credits DataFrames based on common ID
joined_df = titles_df.join(credits_df, titles_df['id'] == credits_df['id'])


Grouping and Aggregation
--
Find the average IMDb score for movies directed by each director.

In [21]:
# Group by director and calculate average IMDb score
average_imdb_scores_df = joined_df.filter(joined_df.type == "MOVIE") \
                                   .groupBy("name") \
                                   .agg({"imdb_score": "avg"})

In [22]:
average_imdb_scores_df.show()


+-------------------+------------------+
|               name|   avg(imdb_score)|
+-------------------+------------------+
|        John Curtis|               8.3|
|        Frank Sisto|               8.3|
|     Ethan Phillips|               7.4|
|    Kottayam Ramesh|               6.5|
|        Rumiko Ukai|               7.4|
|       Johnny Lever|           5.81875|
|     Bob Stephenson|             6.625|
|      Michael Stein|               7.9|
|         Brett Rice|7.8999999999999995|
|       Garth Wilton|               7.9|
|    Stephen Shellen|               7.2|
|Dex Elliott Sanders|               6.4|
|          M.G. Gong|               6.1|
|           Pooh Man|               7.5|
|        Shane Dunne|               6.2|
|         Snoop Dogg|               6.3|
|        Matthew Tal|               7.3|
|        April Grace|               7.2|
|   Christian Stolte|               6.1|
|        Jordi Mollà|               5.6|
+-------------------+------------------+
only showing top

Sorting
---
 Sort the DataFrame by IMDb score in descending order.

In [23]:
# Assuming 'imdb_score' is the column name for IMDb score
sorted_df = titles_df.orderBy(titles_df['imdb_score'].desc())

# Show the sorted DataFrame
sorted_df.show()

26/01/28 15:57:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, type, release_year, age_certification, runtime, seasons, imdb_id, imdb_score, imdb_votes, tmdb_popularity, tmdb_score
 Schema: _c0, id, type, release_year, age_certification, runtime, seasons, imdb_id, imdb_score, imdb_votes, tmdb_popularity, tmdb_score
Expected: _c0 but found: 
CSV file: file:///uufs/chpc.utah.edu/common/home/u1471478/Documents/CS-6964-Lab-2-Do-Not-Share-with-Students/dataset/titles.csv


+----+--------+-----+------------+-----------------+-------+-------+----------+----------+----------+---------------+----------+
| _c0|      id| type|release_year|age_certification|runtime|seasons|   imdb_id|imdb_score|imdb_votes|tmdb_popularity|tmdb_score|
+----+--------+-----+------------+-----------------+-------+-------+----------+----------+----------+---------------+----------+
|2658|ts265844| SHOW|        2018|            TV-PG|     68|    1.0|tt12635254|       9.6|       7.0|           NULL|      NULL|
| 216|     ts4| SHOW|        2008|            TV-MA|     48|    5.0| tt0903747|       9.5| 1775990.0|        353.848|      8.79|
| 564|ts160526| SHOW|        2005|            TV-14|     19|   11.0| tt3062514|       9.5|    3115.0|           NULL|      NULL|
| 233|  ts3371| SHOW|        2005|            TV-Y7|     24|    3.0| tt0417299|       9.3|  303666.0|         56.915|       8.7|
|3147| ts85398| SHOW|        2019|             TV-G|     50|    1.0| tt9253866|       9.3|   4225

In [24]:

top_rated_movies_df = titles_df.orderBy(col("imdb_score").desc()).limit(5)

highest_popularity_movie_df = titles_df.orderBy(col("tmdb_popularity").desc()).limit(1)


# Display results
print("Top Rated shows:")
top_rated_movies_df.show(truncate=False)
print("Highest Popularity Movie:")
highest_popularity_movie_df.show(truncate=False)

Top Rated shows:
+----+--------+----+------------+-----------------+-------+-------+----------+----------+----------+---------------+----------+
|_c0 |id      |type|release_year|age_certification|runtime|seasons|imdb_id   |imdb_score|imdb_votes|tmdb_popularity|tmdb_score|
+----+--------+----+------------+-----------------+-------+-------+----------+----------+----------+---------------+----------+
|2658|ts265844|SHOW|2018        |TV-PG            |68     |1.0    |tt12635254|9.6       |7.0       |NULL           |NULL      |
|564 |ts160526|SHOW|2005        |TV-14            |19     |11.0   |tt3062514 |9.5       |3115.0    |NULL           |NULL      |
|216 |ts4     |SHOW|2008        |TV-MA            |48     |5.0    |tt0903747 |9.5       |1775990.0 |353.848        |8.79      |
|233 |ts3371  |SHOW|2005        |TV-Y7            |24     |3.0    |tt0417299 |9.3       |303666.0  |56.915         |8.7       |
|3147|ts85398 |SHOW|2019        |TV-G             |50     |1.0    |tt9253866 |9.3      

26/01/28 15:57:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, type, release_year, age_certification, runtime, seasons, imdb_id, imdb_score, imdb_votes, tmdb_popularity, tmdb_score
 Schema: _c0, id, type, release_year, age_certification, runtime, seasons, imdb_id, imdb_score, imdb_votes, tmdb_popularity, tmdb_score
Expected: _c0 but found: 
CSV file: file:///uufs/chpc.utah.edu/common/home/u1471478/Documents/CS-6964-Lab-2-Do-Not-Share-with-Students/dataset/titles.csv
26/01/28 15:57:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, type, release_year, age_certification, runtime, seasons, imdb_id, imdb_score, imdb_votes, tmdb_popularity, tmdb_score
 Schema: _c0, id, type, release_year, age_certification, runtime, seasons, imdb_id, imdb_score, imdb_votes, tmdb_popularity, tmdb_score
Expected: _c0 but found: 
CSV file: file:///uufs/chpc.utah.edu/common/home/u1471478/Documents/CS-6964-Lab-2-Do-Not-Share-with-Students/datas

+----+---------+-----+------------+-----------------+-------+-------+---------+----------+----------+---------------+----------+
|_c0 |id       |type |release_year|age_certification|runtime|seasons|imdb_id  |imdb_score|imdb_votes|tmdb_popularity|tmdb_score|
+----+---------+-----+------------+-----------------+-------+-------+---------+----------+----------+---------------+----------+
|4699|tm1020438|MOVIE|2022        |R                |106    |NULL   |tt9783600|5.4       |43239.0   |996.869        |5.8       |
+----+---------+-----+------------+-----------------+-------+-------+---------+----------+----------+---------------+----------+



##Spark SQL

In [25]:
# Register DataFrames as temporary tables
titles_df.createOrReplaceTempView("titles")
credits_df.createOrReplaceTempView("credits")

Join Operation (SQL):

In [26]:
# Join titles and credits tables
joined_df = spark.sql("""
    SELECT *
    FROM titles t
    JOIN credits c
    ON t.id = c.id
""")

Filtering (SQL): Filter the "titles" table to find movies released after 2010 using SQL.

In [27]:
# Filter movies released after 2010
recent_movies_df = spark.sql("""
    SELECT *
    FROM titles
    WHERE release_year > 2010
""")

In [28]:
recent_movies_df.show()

+---+--------+-----+------------+-----------------+-------+-------+---------+----------+----------+---------------+----------+
|_c0|      id| type|release_year|age_certification|runtime|seasons|  imdb_id|imdb_score|imdb_votes|tmdb_popularity|tmdb_score|
+---+--------+-----+------------+-----------------+-------+-------+---------+----------+----------+---------------+----------+
|640| ts20660| SHOW|        2011|            TV-MA|     54|   11.0|tt1586680|       8.6|  234232.0|        252.934|     8.112|
|641| tm68272|MOVIE|        2012|                R|     88|   NULL|tt1636826|       6.6|  207830.0|         148.44|       6.8|
|642| tm61801|MOVIE|        2012|                R|    157|   NULL|tt1790885|       7.4|  296416.0|         24.241|       7.0|
|643| tm44730|MOVIE|        2012|            PG-13|    136|   NULL|tt0948470|       6.9|  646119.0|        159.056|       6.7|
|644| ts21095| SHOW|        2012|            TV-Y7|     24|    5.0|tt1877889|       7.8|   10809.0|         48.

26/01/28 15:57:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, type, release_year, age_certification, runtime, seasons, imdb_id, imdb_score, imdb_votes, tmdb_popularity, tmdb_score
 Schema: _c0, id, type, release_year, age_certification, runtime, seasons, imdb_id, imdb_score, imdb_votes, tmdb_popularity, tmdb_score
Expected: _c0 but found: 
CSV file: file:///uufs/chpc.utah.edu/common/home/u1471478/Documents/CS-6964-Lab-2-Do-Not-Share-with-Students/dataset/titles.csv


Grouping and Aggregation (SQL): Find the average IMDb score for movies directed by each director using SQL.

In [29]:
# Calculate average IMDb score by director
average_imdb_scores_df = spark.sql("""
    SELECT name, AVG(imdb_score) AS avg_imdb_score
    FROM titles t
    JOIN credits c
    ON t.id = c.id
    WHERE t.type = 'MOVIE'
    GROUP BY name
""")


In [30]:
average_imdb_scores_df.show()

+-------------------+------------------+
|               name|    avg_imdb_score|
+-------------------+------------------+
|        John Curtis|               8.3|
|        Frank Sisto|               8.3|
|     Ethan Phillips|               7.4|
|    Kottayam Ramesh|               6.5|
|        Rumiko Ukai|               7.4|
|       Johnny Lever|           5.81875|
|     Bob Stephenson|             6.625|
|      Michael Stein|               7.9|
|         Brett Rice|7.8999999999999995|
|       Garth Wilton|               7.9|
|    Stephen Shellen|               7.2|
|Dex Elliott Sanders|               6.4|
|          M.G. Gong|               6.1|
|           Pooh Man|               7.5|
|        Shane Dunne|               6.2|
|         Snoop Dogg|               6.3|
|        Matthew Tal|               7.3|
|        April Grace|               7.2|
|   Christian Stolte|               6.1|
|        Jordi Mollà|               5.6|
+-------------------+------------------+
only showing top

Spark MLlib
--
The following tutorial will help in understanding classification task using spark MLlib

In [31]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col, when
from statistics import mode as _mode
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import Imputer
from pyspark.sql.window import Window
from pyspark.sql.functions import last
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [32]:
rain_pred = spark.read.csv('dataset/weatherAUS.csv', inferSchema = True, header = True)


In [33]:
rain_pred = rain_pred.drop('Date', 'Location', 'Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm')


In [34]:
rain_pred.show(20)


+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|   13.4|   22.9|     0.6|          W|           44|         W|       WNW|          20|          24|         71|         22|     1007.7|     1007.1|       No|          No|
|    7.4|   25.1|       0|        WNW|           44|       NNW|       WSW|           4|          22|         44|         25|     1010.6|     1007.8|       No|          No|
|   12.9|   25.7|       0|        WSW|           46|         W|       WSW|          19|          26|         38|         30|     1007.6|    

In [35]:
rain_pred.columns


['MinTemp',
 'MaxTemp',
 'Rainfall',
 'WindGustDir',
 'WindGustSpeed',
 'WindDir9am',
 'WindDir3pm',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'RainToday',
 'RainTomorrow']

In [36]:
for columns in rain_pred.columns:
    print(rain_pred.select(_mean(col(columns))).collect())

[Row(avg(MinTemp)=12.194034380968894)]
[Row(avg(MaxTemp)=23.221348275647014)]
[Row(avg(Rainfall)=2.3609181499168272)]
[Row(avg(WindGustDir)=None)]
[Row(avg(WindGustSpeed)=40.03523007167319)]
[Row(avg(WindDir9am)=None)]
[Row(avg(WindDir3pm)=None)]
[Row(avg(WindSpeed9am)=14.043425914971502)]
[Row(avg(WindSpeed3pm)=18.662656778887342)]
[Row(avg(Humidity9am)=68.88083133761887)]
[Row(avg(Humidity3pm)=51.5391158755046)]
[Row(avg(Pressure9am)=1017.6499397983058)]
[Row(avg(Pressure3pm)=1015.2558888309709)]
[Row(avg(RainToday)=None)]
[Row(avg(RainTomorrow)=None)]


Data Transformation
----
Machine Learning Algorithms, often accept only numerical columns as input and would throw an error if supplied with non-numerical variables. Hence, we need to convert the numerical columns from string to double and convert categorical columns from string to numbers that the algorithm can directly use to process.

First, let's make a list of numerical columns and name it numerical_list. Looping over the numerical_list, we use cast(DoubleType()) to convert it from string to double.




In [37]:
numerical_list = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
                  'Humidity3pm', 'Pressure9am', 'Pressure3pm']

In [38]:
for columns in numerical_list:
    rain_pred = rain_pred.withColumn(columns, rain_pred[columns].cast(DoubleType()))

In [39]:
def missing():
    for columns in rain_pred.columns:
        print(columns + ' has number of NULLs : ' + str(rain_pred[rain_pred[columns] == 'NA'].count()))

In [40]:
missing()

MinTemp has number of NULLs : 0
MaxTemp has number of NULLs : 0
Rainfall has number of NULLs : 0
WindGustDir has number of NULLs : 10326
WindGustSpeed has number of NULLs : 0
WindDir9am has number of NULLs : 10566
WindDir3pm has number of NULLs : 4228
WindSpeed9am has number of NULLs : 0
WindSpeed3pm has number of NULLs : 0
Humidity9am has number of NULLs : 0
Humidity3pm has number of NULLs : 0
Pressure9am has number of NULLs : 0
Pressure3pm has number of NULLs : 0
RainToday has number of NULLs : 3261
RainTomorrow has number of NULLs : 3267


In [41]:
rain_pred.show()

+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|   13.4|   22.9|     0.6|          W|         44.0|         W|       WNW|        20.0|        24.0|       71.0|       22.0|     1007.7|     1007.1|       No|          No|
|    7.4|   25.1|     0.0|        WNW|         44.0|       NNW|       WSW|         4.0|        22.0|       44.0|       25.0|     1010.6|     1007.8|       No|          No|
|   12.9|   25.7|     0.0|        WSW|         46.0|         W|       WSW|        19.0|        26.0|       38.0|       30.0|     1007.6|    

In [42]:
rain_pred.printSchema()


root
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Pressure9am: double (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: string (nullable = true)



Categorical Columns
-----
Things will be a bit different here, we shall make use of the .groupBy(), .count(), orderBy() and .sort() methods on our dataset to firstly see, and then change their values in the dataset.

In [43]:
rain_pred.groupBy('WindGustDir').count().orderBy('WindGustDir').sort('count', ascending = False).show()


+-----------+-----+
|WindGustDir|count|
+-----------+-----+
|         NA|10326|
|          W| 9915|
|         SE| 9418|
|          N| 9313|
|        SSE| 9216|
|          E| 9181|
|          S| 9168|
|        WSW| 9069|
|         SW| 8967|
|        SSW| 8736|
|        WNW| 8252|
|         NW| 8122|
|        ENE| 8104|
|        ESE| 7372|
|         NE| 7133|
|        NNW| 6620|
|        NNE| 6548|
+-----------+-----+



In [44]:
rain_pred.groupBy('WindDir9am').count().orderBy('WindDir9am').sort('count', ascending = False).show()


+----------+-----+
|WindDir9am|count|
+----------+-----+
|         N|11758|
|        NA|10566|
|        SE| 9287|
|         E| 9176|
|       SSE| 9112|
|        NW| 8749|
|         S| 8659|
|         W| 8459|
|        SW| 8423|
|       NNE| 8129|
|       NNW| 7980|
|       ENE| 7836|
|        NE| 7671|
|       ESE| 7630|
|       SSW| 7587|
|       WNW| 7414|
|       WSW| 7024|
+----------+-----+



String Indexer
----
This is a feature transformer from Spark MLlib that converts a column of labels into a column of label indices. These indices are ordered by label frequencies, so the most frequent label gets index 0. The inputCol parameter specifies the name of the column to be indexed, and the outputCol parameter specifies the name of the new column that will contain the numerical indices.

In [45]:
categorical_list = ['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']

indexers = [StringIndexer(inputCol = column, outputCol = column + "_index").fit(rain_pred) for column in categorical_list]
pipeline = Pipeline(stages = indexers)
rain_pred_indexers = pipeline.fit(rain_pred).transform(rain_pred)
rain_pred_indexers.show(10)

+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+-----------------+----------------+----------------+---------------+------------------+
|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|WindGustDir_index|WindDir9am_index|WindDir3pm_index|RainToday_index|RainTomorrow_index|
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+-----------------+----------------+----------------+---------------+------------------+
|   13.4|   22.9|     0.6|          W|         44.0|         W|       WNW|        20.0|        24.0|       71.0|       22.0|     1007.7|     1007.1|       No|          No|              1.0|             7.0|             7.0|

In [46]:
drop = ['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']

rain_pred_indexers = rain_pred_indexers.drop(*drop)

rain_pred_indexers.show(10)

+-------+-------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+-----------------+----------------+----------------+---------------+------------------+
|MinTemp|MaxTemp|Rainfall|WindGustSpeed|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|WindGustDir_index|WindDir9am_index|WindDir3pm_index|RainToday_index|RainTomorrow_index|
+-------+-------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+-----------------+----------------+----------------+---------------+------------------+
|   13.4|   22.9|     0.6|         44.0|        20.0|        24.0|       71.0|       22.0|     1007.7|     1007.1|              1.0|             7.0|             7.0|            0.0|               0.0|
|    7.4|   25.1|     0.0|         44.0|         4.0|        22.0|       44.0|       25.0|     1010.6|     1007.8|             10.0|            10.0|             3.0|            0.0|          

In [47]:
missing()

MinTemp has number of NULLs : 0
MaxTemp has number of NULLs : 0
Rainfall has number of NULLs : 0
WindGustDir has number of NULLs : 10326
WindGustSpeed has number of NULLs : 0
WindDir9am has number of NULLs : 10566
WindDir3pm has number of NULLs : 4228
WindSpeed9am has number of NULLs : 0
WindSpeed3pm has number of NULLs : 0
Humidity9am has number of NULLs : 0
Humidity3pm has number of NULLs : 0
Pressure9am has number of NULLs : 0
Pressure3pm has number of NULLs : 0
RainToday has number of NULLs : 3261
RainTomorrow has number of NULLs : 3267


Imputer
----
Replacing missing numerical values with the mean of the column.

In [48]:
imputer = Imputer(strategy="mean", inputCols=numerical_list, outputCols=numerical_list)
model = imputer.fit(rain_pred_indexers)

rain_pred_indexers = model.transform(rain_pred_indexers)

 Create the Feature Vector and Divide the Dataset
----
Here, we are required to use the VectorAssembler() to convert the set of features into a feature vector.

In [49]:
target = ['RainTomorrow_index']

assembler = VectorAssembler(
    inputCols = [columns for columns in rain_pred_indexers.columns if columns not in target],
    outputCol = 'feature_vector')

rain_pred_indexers_data = assembler.transform(rain_pred_indexers)

In [50]:
rain_test, rain_train = rain_pred_indexers_data.randomSplit([0.3, 0.7], seed = 123)


Random Forest
----
We shall now build Random Forest. Using the RandomForestClassifier() and .fit() method on our training data.

In [51]:
rain_pred_indexers_data.show(10)


26/01/28 15:58:16 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+-----------------+----------------+----------------+---------------+------------------+--------------------+
|MinTemp|MaxTemp|Rainfall|WindGustSpeed|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|WindGustDir_index|WindDir9am_index|WindDir3pm_index|RainToday_index|RainTomorrow_index|      feature_vector|
+-------+-------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+-----------------+----------------+----------------+---------------+------------------+--------------------+
|   13.4|   22.9|     0.6|         44.0|        20.0|        24.0|       71.0|       22.0|     1007.7|     1007.1|              1.0|             7.0|             7.0|            0.0|               0.0|[13.4,22.9,0.6,44...|
|    7.4|   25.1|     0.0|         44.0|         4.0|        22.0|       44.0|       25.0|     1010.6|     1

In [52]:
RF_Classifier = RandomForestClassifier(labelCol = 'RainTomorrow_index', featuresCol = 'feature_vector',  maxDepth = 5,
    maxBins = 32, numTrees = 500)

RF_Model = RF_Classifier.fit(rain_train)

26/01/28 15:58:53 WARN DAGScheduler: Broadcasting large task binary with size 1279.6 KiB
26/01/28 15:59:06 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB


In [53]:
RF_Prediction = RF_Model.transform(rain_test)


In [54]:
RF_Prediction.head(5)


26/01/28 15:59:27 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


[Row(MinTemp=-8.5, MaxTemp=0.6, Rainfall=1.0, WindGustSpeed=31.0, WindSpeed9am=20.0, WindSpeed3pm=15.0, Humidity9am=47.0, Humidity3pm=72.0, Pressure9am=1017.6499397983058, Pressure3pm=1015.2558888309709, WindGustDir_index=6.0, WindDir9am_index=15.0, WindDir3pm_index=5.0, RainToday_index=0.0, RainTomorrow_index=0.0, feature_vector=DenseVector([-8.5, 0.6, 1.0, 31.0, 20.0, 15.0, 47.0, 72.0, 1017.6499, 1015.2559, 6.0, 15.0, 5.0, 0.0]), rawPrediction=DenseVector([366.0343, 126.7582, 7.2074]), probability=DenseVector([0.7321, 0.2535, 0.0144]), prediction=0.0),
 Row(MinTemp=-8.0, MaxTemp=15.1, Rainfall=0.2, WindGustSpeed=40.03523007167319, WindSpeed9am=6.0, WindSpeed3pm=13.0, Humidity9am=96.0, Humidity3pm=33.0, Pressure9am=1029.6, Pressure3pm=1025.3, WindGustDir_index=0.0, WindDir9am_index=10.0, WindDir3pm_index=8.0, RainToday_index=0.0, RainTomorrow_index=0.0, feature_vector=DenseVector([-8.0, 15.1, 0.2, 40.0352, 6.0, 13.0, 96.0, 33.0, 1029.6, 1025.3, 0.0, 10.0, 8.0, 0.0]), rawPrediction=Den

In [55]:
RF_Evaluator = MulticlassClassificationEvaluator(labelCol = 'RainTomorrow_index', predictionCol = 'prediction')


In [56]:
RF_Accuracy = RF_Evaluator.evaluate(RF_Prediction)


26/01/28 15:59:29 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


In [57]:
print('Accuracy is : ' + str(RF_Accuracy * 100))
print('Test Error is : ' + str(1 - RF_Accuracy))

Accuracy is : 80.00387072213219
Test Error is : 0.19996129277867802
